In [1]:
import torch
from torch import nn
import random, numpy
import numpy as np
from copy import deepcopy
from game import Game, Move, RandomPlayer, MyPlayer,translate_number_to_position_direction,translate_number_to_position,TrainedPlayer
import torch.nn.init as init

In [2]:
class Genome:
    def __init__(self, ):
        self.genotype = create_rand_gen()
        self.fitness = float("-inf")

cell_max_values = [
    (1, 8),    # num_matches
    (100, 1_000),  # max_dim_replay_buff
    (2, 80),   # time_to_update
    (0.2, 1.0),   # gamma
    (50, 500)    # batch_size
]

def create_rand_gen(): # i create the genome referred to the scale of our values we'd like to estimate    
    # Define the maximum values for each cell
    # Generate a random vector
    genome = [random.uniform(cell[0], cell[1]) if isinstance(cell[0], float) else random.randint(cell[0], cell[1]) for cell in cell_max_values]

    genome[4] = min(genome[1], genome[4]) # faccio in modo che max_dim_replay_buff > batch_size

    return genome


In [3]:
class ReplayBuffer:    
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size        
        self.buffer = []
        self.position = 0
        
    def add_experience(self, experience):        
        if len(self.buffer) < self.buffer_size:
            self.buffer.append(experience)        
        else:
            self.buffer[self.position] = experience        
            self.position = (self.position + 1) % self.buffer_size
            
    def sample_batch(self, batch_size):
        batch_indices = np.random.choice(len(self.buffer), batch_size, replace=True)
        batch = [self.buffer[i] for i in batch_indices]
        return batch

In [4]:
from game import Player
from tqdm import tqdm


def training(individual: 'Genome'):
    ## Constants definition
    num_iterations = 200
    num_matches = individual.genotype[0]
    max_dim_replay_buff = individual.genotype[1]
    time_to_update = individual.genotype[2]
    gamma = individual.genotype[3]
    batch_size = individual.genotype[4]
    # potentially also the learning rate
    player1 = MyPlayer()
    player2 = RandomPlayer()

    loss_tot = []
    
    taboo_set = set()
    replay_buff = ReplayBuffer(max_dim_replay_buff) # replay buffer, from which we sample for BATCH learning
    # inizializza i modelli -> passali per parametro
    torch.set_grad_enabled(True)
    

    
    for step in range(num_iterations):  
        # we fill the replay buffer with experiences made with matches
        #player1.step=step
        for match in range(num_matches):
            # lo uso solo in inferenza
            # gioca una intera partita qui dentro e per ogni mossa, metti l'experience dentro il ReplayBuffer
            # ...
            # ... qui descrivere come viene effettivamente usata la rete neurale (magari impacchettarla in qualche funzione)
            winner = -1
            g = Game()
            #g.current_player_idx = match%2
            g.current_player_idx = 1

            #player1 = MyPlayer()
            #player2 = RandomPlayer()
            players = [player1, player2]

            go = False # serve a calcolare lo stato successivo per il calcolo del target

            while winner < 0:
                    g.current_player_idx += 1
                    g.current_player_idx %= len(players)
                    prev_state=deepcopy(g)
                    from_pos, slide = players[g.current_player_idx].make_move(g)
                    
                    if g.current_player_idx==0:
                        ##l'azione è il valore del q-value quindi un numero? o la posizione in questo caso
                        ##in quel caso replay_row=(prev_state.get_board,GeneratorNet((prev_state.get_board),g.get_board,reward)
                        reward=g.compute_reward(from_pos, slide)
                        go = True
                        #io qua dentro x ogni row voglio i 44 output della mia rete
                        
                        #replay_row=(prev_state.get_flat_board(),player1.last_action_number, deepcopy(g), reward)

                        # faccio fare una mossa all'avversario e poi faccio il max action sul nuovo stato che mi ritorna
                        
                    elif go and g.current_player_idx==1:

                        #io qua dentro x ogni row voglio i 44 output della mia rete
                        if (tuple(prev_state.get_flat_board()), player1.last_action_number, tuple(deepcopy(g).get_flat_board())) not in taboo_set :
                            replay_row=(prev_state.get_flat_board(), player1.last_action_number, deepcopy(g), reward, reward==1)
                            taboo_set.add((tuple(prev_state.get_flat_board()),player1.last_action_number, tuple(deepcopy(g).get_flat_board())))
                            #eliminato last_action da replay row?? Non dovrebbe servire
                            replay_buff.add_experience(replay_row)

                        #replay_buff.add_experience(replay_row)                        

                    if g.check_winner() != -1:
                        break
            
            # replay_buffer.push(...) per ogni transizione di stato osservata ()
            # restituisci il risultato sotto forma di tupla (st, a, st+1, r) - (State, action, next_state, reward)
            
        # Now we sample a batch of data from the ReplayBuffer in order to train the Agent
        batch_to_train = replay_buff.sample_batch(batch_size)
      
        #divide the batch
        state_batch, action_num, next_state_batch, reward, done = zip(*batch_to_train) 

        #forward the Generator
        q_values = player1.GeneratorNet(torch.tensor(state_batch, dtype=torch.float32)).to(player1.device)
        q_values_target = torch.zeros(batch_size, 44).to(player1.device)


        #update q_values target by using Bellman Equation  
        for i in range(batch_size): 
            if done[i] == False:
                q_values_target[i, action_num[i]] = reward[i] + gamma * torch.tensor(player1.compute_target(next_state_batch[i])).to(player1.device).item()
            else:
               q_values_target[i, action_num[i]] = reward[i] 
          
        player1.optimizer.zero_grad()
        loss_curr=player1.criterion(q_values,q_values_target).to(player1.device)
        loss_curr.backward()
    
        player1.optimizer.step()

        loss_tot.append(loss_curr)
      
        if (step % time_to_update) == 0:
            # update the parameter of the TargetNet
            player1.copy_params_TargetNet()

        # si dovrebbe aggiungere alla struttura del Genoma anche il dizionario ed la loss_tot
            
    return  player1.GeneratorNet.state_dict()
            
      

In [5]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 4
LOCI = 5
MUTATION_PROBABILITY = 0.5
BIT_MUTATION_PROBABILITY = 0.5
NUM_GENERATION = 50

In [6]:
from random import randint
from copy import deepcopy

import tqdm


def new_offspring(parent1: 'Genome', parent2: 'Genome') -> 'Genome':
    if random.random() < MUTATION_PROBABILITY:
        return mutate(parent1)
    return three_cut_xover(parent1, parent2)
    
def mutate(parent: 'Genome') -> 'Genome':
    new_offspring = deepcopy(parent)

    # Apply bit flip to the shuffled elements
    for _ in range(LOCI):
        if random.random() < BIT_MUTATION_PROBABILITY:
            index_to_mutate = randint(0, LOCI-1)
            new_offspring.genotype[index_to_mutate] = random.uniform(cell_max_values[index_to_mutate][0], cell_max_values[index_to_mutate][1]) if isinstance(cell_max_values[index_to_mutate][0], float) else random.randint(cell_max_values[index_to_mutate][0], cell_max_values[index_to_mutate][1])
            new_offspring.genotype[4] = min(new_offspring.genotype[1], new_offspring.genotype[4]) # faccio in modo che max_dim_replay_buff > batch_size
    
    return new_offspring

def three_cut_xover(ind1: 'Genome', ind2: 'Genome') -> 'Genome':
    one_cut_point = randint(0, int((LOCI)*0.3))
    two_cut_point = randint(int((LOCI)*0.3), int((LOCI)*0.6))
    three_cut_point = randint(int((LOCI)*0.6), LOCI - 1)
  
    # Order the cut points
    cut_points = sorted([one_cut_point, two_cut_point, three_cut_point])
    
    new_ind = Genome()
    new_ind.genotype = (ind1.genotype[:cut_points[0]] +
                        ind2.genotype[cut_points[0]:cut_points[1]] +
                        ind1.genotype[cut_points[1]:cut_points[2]] +
                        ind2.genotype[cut_points[2]:])
    
    new_ind.genotype[4] = min(new_ind.genotype[1], new_ind.genotype[4]) # faccio in modo che max_dim_replay_buff > batch_size
    
    assert len(new_ind.genotype) == LOCI
    return new_ind

def compute_fitness(individual: 'Genome'):
    TrainedGeneratorNet = TrainedPlayer()
    trained_weights = training(individual)
    TrainedGeneratorNet.GeneratorNet.load_state_dict(trained_weights) # TRAINED NETWORK to use in INFERENCE PHASE

    wins = 0
    num_match_test = 100

    for step in range(num_match_test):
        player=RandomPlayer()
        g=Game()
        #g.current_player_idx = step%2
        g.current_player_idx += 1
        winner=g.play(TrainedGeneratorNet, player)
        if winner==0:
            wins+=1

    individual.fitness = (wins/num_match_test)*100
    

In [7]:

def genetic_algorithm():
        population = [Genome() for _ in range(POPULATION_SIZE)] # starting pouplation of POPULATION_SIZE individuals
        for i in range(OFFSPRING_SIZE):
                compute_fitness(population[i])

        population.sort(key=lambda i: i.fitness, reverse=True)
        best_fitness = population[0].fitness
        best_individual = population[0]
        gen = 0

        while  gen < NUM_GENERATION:                
                for _ in range(OFFSPRING_SIZE):
                        offspring = new_offspring(population[0], population[1])
                        compute_fitness(offspring)

                population.extend([offspring])
                population.sort(key=lambda i: i.fitness, reverse=True)
                population = population[:POPULATION_SIZE]
                best_individual = population[0]
                best_fitness = population[0].fitness
                gen += 1
                print(f"Best individual (until now): {best_individual.genotype}")
                print(f"Best fitness: {best_fitness}")


        print(f"Best individual with fitness: {best_fitness}")
        
        return best_individual



In [8]:
BEST_INDIVIDUAL = genetic_algorithm()

C:\Users\chito\AppData\Local\Temp\ipykernel_17660\2825236233.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  q_values = player1.GeneratorNet(torch.tensor(state_batch, dtype=torch.float32)).to(player1.device)


Best individual (until now): [2, 778, 30, 0.4850682937735741, 178]
Best fitness: 60.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 178]
Best fitness: 60.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 178]
Best fitness: 60.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 178]
Best fitness: 60.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937735741, 351]
Best fitness: 73.0
Best individual (until now): [2, 778, 30, 0.4850682937

In [9]:
from matplotlib import pyplot as plt

vals = [el.detach().numpy() for el in BEST_INDIVIDUAL.loss_tot]
# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, len(BEST_INDIVIDUAL.loss_tot) + 1)

# Plot and label the training and validation loss values
plt.plot(epochs, vals, label='Training Loss')

# Add title and axes labels
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Display the plot
plt.legend()
plt.show()

AttributeError: 'Genome' object has no attribute 'loss_tot'